In [14]:
from gen_ai_hub.proxy.core.proxy_clients import set_proxy_version

#use btp proxy llm


from gen_ai_hub.proxy.langchain import init_llm
from generative_ai_toolkit_for_sap_hana_cloud.vectorstore.embedding_service import GenAIHubEmbeddings
llm = init_llm('gpt-4', temperature=0.0, max_tokens=512)
embed_model = GenAIHubEmbeddings()


In [15]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [17]:
import os
from llama_index.core import VectorStoreIndex
from llama_index.readers.github import GithubRepositoryReader, GithubClient
from IPython.display import Markdown, display

gh_token = os.getenv("GITHUB_TOKEN")
client = GithubClient(gh_token)
loader = GithubRepositoryReader(
        github_client=client,
        owner='SAP-samples',
        repo='hana-ml-samples',
        use_parser=False,
        verbose=True,
        timeout=None,
        retries=2,
        filter_directories=(
            ["PAL-SQL"],
            GithubRepositoryReader.FilterType.INCLUDE,
        ),
        filter_file_extensions=(
            [
                ".sql",
            ],
            GithubRepositoryReader.FilterType.INCLUDE,
        ),
    )


In [18]:
import nest_asyncio
nest_asyncio.apply()

In [19]:
docs = loader.load_data(branch='main')


current path: 
tree data: GitTreeResponseModel(sha='c816012e591bea3e9af863cdafbc72b60ef55404', url='https://api.github.com/repos/SAP-samples/hana-ml-samples/git/trees/c816012e591bea3e9af863cdafbc72b60ef55404', tree=[GitTreeResponseModel.GitTreeObject(path='.reuse', mode='040000', type='tree', sha='8fb8459605a121e272dd499eeacbe3b32d39c9cd', url='https://api.github.com/repos/SAP-samples/hana-ml-samples/git/trees/8fb8459605a121e272dd499eeacbe3b32d39c9cd', size=None), GitTreeResponseModel.GitTreeObject(path='.vscode', mode='040000', type='tree', sha='5d858526e1cf187dc94424cd8869ded4f0507f2e', url='https://api.github.com/repos/SAP-samples/hana-ml-samples/git/trees/5d858526e1cf187dc94424cd8869ded4f0507f2e', size=None), GitTreeResponseModel.GitTreeObject(path='APL-SQL', mode='040000', type='tree', sha='4e94dfec4a3f592c23dbc8e59f363e23238c14c9', url='https://api.github.com/repos/SAP-samples/hana-ml-samples/git/trees/4e94dfec4a3f592c23dbc8e59f363e23238c14c9', size=None), GitTreeResponseModel.Gi

In [20]:
index = VectorStoreIndex.from_documents(docs)

In [24]:
question = "How to use random decision trees in PAL sql?"

In [25]:
chat_engine = index.as_chat_engine(chat_mode="context", verbose=True)

response = chat_engine.chat(question)

In [26]:
print(response)

Random Decision Trees is a machine learning algorithm used for classification and regression tasks. It creates multiple decision trees and aggregates their results. Here's a simplified step-by-step guide on how to use Random Decision Trees in PAL SQL:

1. **Prepare your data**: Your data should be in a table with one column for each feature and one column for the target variable. The target variable should be binary for classification tasks.

2. **Create a parameter table**: This table will hold the parameters for the Random Decision Trees algorithm. Some of the parameters you might set include the number of trees to create (`TREES_NUM`), the maximum depth of each tree (`MAX_DEPTH`), and the minimum number of records in a leaf (`NODE_SIZE`).

3. **Call the Random Decision Trees algorithm**: Use the `_SYS_AFL.PAL_RANDOM_DECISION_TREES` procedure to train the model. Pass in your data table, parameter table, and empty tables to hold the output.

   Example:
   ```
   CALL _SYS_AFL.PAL_RAN